# Road Follower - Train Model



In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

In [ ]:
!unzip -q road_following.zip

You should see a folder named ``dataset_all`` appear in the file browser.

### Create Dataset Instance



In [ ]:
def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[0])) - width/2) / (width/2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[1])) - height/2) / (height/2)

class XYDataset(torch.utils.data.Dataset):

    def __init__(self, directory, random_hflips=True):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))

        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x

        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

        return image, torch.tensor([x, y]).float()

dataset = XYDataset('/content/sample_data/road_following', random_hflips=False)

In [ ]:
dataset.__getitem__(0)

(tensor([[[ 0.7077,  0.8447,  0.8276,  ...,  0.3309,  0.3481,  0.3823],
          [ 0.7077,  0.8276,  0.8104,  ...,  0.3481,  0.3481,  0.3309],
          [ 0.7248,  0.8276,  0.8104,  ...,  0.3481,  0.3309,  0.2967],
          ...,
          [-0.5424, -0.5596, -0.5082,  ..., -0.8678, -0.8678, -0.7993],
          [-0.4911, -0.5596, -0.5424,  ..., -0.8507, -0.8164, -0.7822],
          [-0.4226, -0.4739, -0.5082,  ..., -0.8849, -0.8507, -0.8335]],
 
         [[ 0.4678,  0.6078,  0.5903,  ...,  0.3627,  0.3627,  0.4153],
          [ 0.4678,  0.5728,  0.5728,  ...,  0.3627,  0.3627,  0.3627],
          [ 0.4853,  0.5728,  0.5728,  ...,  0.3627,  0.3627,  0.3277],
          ...,
          [-0.5826, -0.6176, -0.5476,  ..., -0.8102, -0.7927, -0.7227],
          [-0.5301, -0.6352, -0.5826,  ..., -0.7927, -0.7402, -0.7052],
          [-0.4951, -0.5126, -0.5476,  ..., -0.8277, -0.7752, -0.7577]],
 
         [[ 1.8383,  1.9777,  2.0300,  ...,  1.3851,  1.4025,  1.4374],
          [ 1.8383,  1.9603,

### Split dataset into train and test sets
Once we read dataset, we will split data set in train and test sets. In this example we split train and test a 90%-10%. The test set will be used to verify the accuracy of the model we train.

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

In [ ]:
train_dataset.__getitem__(1)

(tensor([[[ 0.0569,  0.0398,  0.0741,  ...,  1.1700,  0.6563, -0.0458],
          [ 0.0569,  0.0398,  0.0569,  ...,  1.1529,  0.8961,  0.0056],
          [ 0.0398,  0.0398,  0.0398,  ...,  1.2385,  1.1358,  0.0398],
          ...,
          [-0.7822, -0.7137, -0.6452,  ..., -0.5938, -0.5938, -0.6281],
          [-0.7137, -0.7137, -0.6109,  ..., -0.5938, -0.6281, -0.5938],
          [-0.7137, -0.6281, -0.6281,  ..., -0.6281, -0.6794, -0.5938]],
 
         [[ 0.6954,  0.7129,  0.7304,  ...,  1.5182,  1.4657,  0.6604],
          [ 0.6954,  0.7129,  0.7129,  ...,  1.5182,  1.4657,  0.6954],
          [ 0.7129,  0.7129,  0.7129,  ...,  1.5532,  1.5182,  0.6779],
          ...,
          [-0.3725, -0.2675, -0.2325,  ..., -0.1275, -0.1275, -0.1800],
          [-0.3025, -0.2850, -0.1975,  ..., -0.1275, -0.1625, -0.1275],
          [-0.3025, -0.2500, -0.2325,  ..., -0.1625, -0.2150, -0.1275]],
 
         [[ 0.5834,  0.6008,  0.6182,  ...,  1.8034,  1.6814,  0.9668],
          [ 0.5834,  0.6008,

### Create data loaders to load data in batches

We use ``DataLoader`` class to load data in batches, shuffle data and allow using multi-subprocesses. In this example we use batch size of 64. Batch size will be based on memory available with your GPU and it can impact accuracy of the model.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

### Define Neural Network Model


In [ ]:
model = models.resnet34(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:04<00:00, 18.0MB/s]


ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1

Finally, we transfer our model for execution on the GPU

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

### Train Regression:

We train for 128 epochs and save best model if the loss is reduced.

In [ ]:
NUM_EPOCHS = 128
BEST_MODEL_PATH = 'best_steering_model_xy_resnet34.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):

    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)

    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)

    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

0.362684, 3.634238
0.046558, 0.108073
0.046285, 0.049332
0.046538, 0.077803
0.049761, 0.048316
0.042374, 0.083923
0.036310, 0.058541
0.042996, 0.070533
0.044609, 0.663793
0.049235, 0.058433
0.041492, 0.066871
0.039952, 0.050553
0.033130, 0.059794
0.030449, 0.050611
0.040957, 0.056327
0.028264, 0.058429
0.035377, 0.117736
0.031974, 0.066163
0.032475, 0.076066
0.034096, 0.064949
0.033460, 0.076201
0.035110, 0.085631
0.042439, 0.057268
0.037849, 0.084455
0.031720, 0.060031
0.030640, 0.055386
0.030914, 0.044305
0.029916, 0.074270
0.030316, 0.052997
0.026362, 0.068530
0.028430, 0.062915
0.028864, 0.064330
0.027196, 0.070942
0.026747, 0.068407
0.026055, 0.064152
0.025494, 0.055487
0.027124, 0.052963
0.024731, 0.054004
0.027424, 0.065252
0.025094, 0.058245
0.024122, 0.066426
0.025415, 0.080543
0.025099, 0.059544
0.027787, 0.058061
0.025721, 0.073814
0.023806, 0.067620
0.028325, 0.074456
0.026848, 0.066154
0.025620, 0.062824
0.025451, 0.075221
0.029414, 0.091437
0.026046, 0.078287
0.023778, 0.

In [ ]:
!pip install torch2trt

ERROR: Could not find a version that satisfies the requirement torch2trt (from versions: none)
ERROR: No matching distribution found for torch2trt


Once the model is trained, it will generate ``best_steering_model_xy.pth`` file which you can use for inferencing in the live demo notebook.

If you trained on a different machine other than JetBot, you'll need to upload this to the JetBot to the ``road_following`` example folder.